# Mistplay Fraud Demo – Model Serving

This notebook shows how to call a Databricks Model Serving endpoint for real-time fraud scoring. Create an endpoint for the latest registered model (`mistplay_fraud_model`) in the UI, then run the request below.

**Suggested endpoint name:** `mistplay-fraud-serving`

**Payload tip:** because the model was logged with Feature Store, you only need to send the entity keys (`account_id`, `device_id`) and Feature Store will perform the feature lookup automatically.

In [ ]:
import os
import requests

DB_NAME = "mistplay_fraud_demo"
ENDPOINT_NAME = "mistplay-fraud-serving"

# Build a small request payload from existing entities
sample_entities = (
    spark.table(f"{DB_NAME}.account_device")
    .select("account_id", "device_id")
    .limit(5)
    .toPandas()
)

payload = {"dataframe_records": sample_entities.to_dict(orient="records")}

# Set these in your environment or replace directly
host = os.environ.get("DATABRICKS_HOST", "https://<your-workspace-host>")
token = os.environ.get("DATABRICKS_TOKEN", "<personal-access-token>")

url = f"{host}/serving-endpoints/{ENDPOINT_NAME}/invocations"
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

response = requests.post(url, headers=headers, json=payload, timeout=30)

print("Status:", response.status_code)
print("Response:", response.text)